In [10]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
pd.options.display.max_info_rows=550
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor

2022-09-12 09:40:09.732021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 09:40:09.732102: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
file = open('../carbon_forecast/pickle/xgboost_d5_model', 'rb')
loaded = pickle.load(file)

In [11]:
from carbon_forecast.utils.preproc import load_n_preproc_all_avail
df = load_n_preproc_all_avail()
X = df.drop(columns=["company_name", "TCUID", "year", "intensity_1and2","intensity_scope3"])
y = df["intensity_1and2"]

Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filli

In [12]:
num_col = sorted(X.select_dtypes(include=["int64", "float64"]).columns)
cat_col = sorted(list(set(X.columns) - set(num_col)))


num_transformer = MinMaxScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, num_col),
    ('cat_tr', cat_transformer, cat_col)
],
sparse_threshold=0)
preprocessor

ColumnTransformer(sparse_threshold=0,
                  transformers=[('num_tr', MinMaxScaler(),
                                 ['Abrasive product manufacturing',
                                  'Accounting, tax preparation, bookkeeping, '
                                  'and payroll services',
                                  'Adhesive manufacturing',
                                  'Advertising and related services',
                                  'Air and gas compressor manufacturing',
                                  'Air conditioning, refrigeration, and warm '
                                  'air heating equipment manufacturing',
                                  'Air purification...
                                  'All other textile product mills',
                                  'All other transportation equipment '
                                  'manufacturing',
                                  'Alumina refining and primary aluminum '
                                  'production',
                                  'Aluminum product manufacturing from '
                                  'purchased aluminum',
                                  'Ammunition manufacturing',
                                  'Amusement parks, arcades, and gambling '
                                  'industries',
                                  'Analytical laboratory instrument '
                                  'manufacturing', ...]),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sector'])])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [20]:
loaded.fit(X_train_transformed, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='mape', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1500,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [21]:
loaded.predict(X_test_transformed)

array([-3.9245024,  8.899129 ,  5.641958 , ..., 19.198616 , 43.95374  ,
       71.57716  ], dtype=float32)

In [31]:
file = open('xgboost_loaded2.pkl', 'wb')
pickle.dump(loaded, file)
file.close()

In [27]:
X_to_test2 = X_test_transformed[2,:]

In [30]:
pd.DataFrame(X_to_test2).to_csv("X_2.csv", index=False)

In [32]:
from xgboost import Booster

In [33]:
loaded.save_model("test.json")

In [34]:
loaded

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='mape', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1500,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [35]:
import joblib
joblib.dump(loaded, "test3.sav")

['test3.sav']